In [1]:
import pandas as pd
import numpy as np

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_12192\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Сокращение дисперсии

Чем меньше дисперсия (а с ней и среднеквадратичное отклонение):
* Тем меньший размер выборки нужен для наблюдения фиксированного эффекта
* Замечать более низкие эффекты

### Увеличение объёма выборки

### Одинаковый (50/50) баланс веток

### Удаление наблюдений.

Причины:
* Значительные выбросы
* Exposured-пользователи, то есть, пользователи из самого начала эксперимента. В тестовую группу могли попасть пользователи, не умеющие работать с тестируемой фичой, они вносят дополнительную дисперсию.

### Стратификация и пост-стратификация

Заранее или во время экспериментов пользователи разбиваются на k **страт** по дополнительным фичам (типу устройства, региону и т.п.).

In [4]:
adsmart_data = pd.read_csv("../data/ad_smart.csv")
adsmart_data = adsmart_data[(adsmart_data.yes == 1) | (adsmart_data.no == 1)].drop(columns=['no'])
adsmart_data

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0
16,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,6,Chrome Mobile,1
20,00a1384a-5118-4d1b-925b-6cdada50318d,exposed,2020-07-06,8,Generic Smartphone,6,Chrome Mobile,0
23,00b6fadb-10bd-49e3-a778-290da82f7a8d,control,2020-07-08,4,Samsung SM-A202F,6,Facebook,1
27,00ebf4a8-060f-4b99-93ac-c62724399483,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0
...,...,...,...,...,...,...,...,...
8059,ffa08ff9-a132-4051-aef5-01a9c79367bc,exposed,2020-07-05,21,Generic Smartphone,6,Chrome Mobile,1
8063,ffb176df-ecd2-45d3-b05f-05b173a093a7,exposed,2020-07-04,1,Generic Smartphone,6,Chrome Mobile,1
8064,ffb79718-6f25-4896-b6b3-e58b80a6e147,control,2020-07-09,7,Generic Smartphone,6,Chrome Mobile,0
8069,ffca1153-c182-4f32-9e90-2a6008417497,control,2020-07-10,16,Generic Smartphone,6,Chrome Mobile,0


In [15]:
adsmart_data.groupby(['experiment', 'platform_os', 'browser']).yes.agg(["sum", "count"])

sum  count
experiment platform_os browser                               
control    5           Chrome Mobile iOS             1      1
                       Mobile Safari                 3      9
                       Mobile Safari UI/WKWebView    0      3
           6           Chrome                        0      1
                       Chrome Mobile               144    324
                       Chrome Mobile WebView        18     47
                       Facebook                     53    112
                       Mobile Safari                 0      1
                       Samsung Internet             45     88
exposed    5           Mobile Safari                 1      4
           6           Chrome                        1      1
                       Chrome Mobile               170    371
                       Chrome Mobile WebView        98    180
                       Facebook                     16     44
                       Samsung Internet             22     57

В каждой страте считается взвешенная оценка дисперсии: 
* $var_{strat}(Ŷ_{strat})=\frac{1}{n}\sum_{i=1}^{k}p_i \sigma_i^2$
* $p_i$ — вероятность попадания в страту (вес)
* $\sigma_i^2$ — дисперсия по страте.<br>

In [37]:
A = adsmart_data[adsmart_data.experiment == 'control'].drop(columns=['experiment'])

def weight(x):
    return len(x) / len(A)

A_strats = A.groupby(['platform_os', 'browser']).yes.agg(["sum", "count", "mean", "var", weight])
A_strats

sum  count      mean       var  \
platform_os browser                                                      
5           Chrome Mobile iOS             1      1  1.000000       NaN   
            Mobile Safari                 3      9  0.333333  0.250000   
            Mobile Safari UI/WKWebView    0      3  0.000000  0.000000   
6           Chrome                        0      1  0.000000       NaN   
            Chrome Mobile               144    324  0.444444  0.247678   
            Chrome Mobile WebView        18     47  0.382979  0.241443   
            Facebook                     53    112  0.473214  0.251528   
            Mobile Safari                 0      1  0.000000       NaN   
            Samsung Internet             45     88  0.511364  0.252743   

                                          weight  
platform_os browser                               
5           Chrome Mobile iOS           0.001706  
            Mobile Safari               0.015358  
            Mobile Safari UI/WKWebView  0.005119  
6           Chrome                      0.001706  
            Chrome Mobile               0.552901  
            Chrome Mobile WebView       0.080205  
            Facebook                    0.191126  
            Mobile Safari               0.001706  
            Samsung Internet            0.150171

За счёт избавления от дисперсии между стратами суммарная дисперсия сокращается.

In [42]:
strat_var = (A_strats['var'] * A_strats['weight']).dropna().sum()
A.yes.var() - strat_var

0.001799941852853315

При этом среднее стратифицированное будет равно среднему по ГС: $E_{strat}(Ŷ_{strat})=\sum_{i=1}^{k}p_i E_{strat}(\overline Y_i) = \sum_{i=1}^{k}p_i \mu_i = \mu$

In [44]:
A.yes.mean() - (A_strats['mean'] * A_strats['weight']).sum()

-5.551115123125783e-17

Отсюда неизменность знака lift'а (оценка среднего не будет отличаться по знаку)

Плохо подобранные переменные для стратификации могут увеличить дисперсию:

In [61]:
bad_strats = 0
c = 20
for i in range(20):
    A['random_strats'] = np.random.randint(0, len(A_strats), size=len(A))
    random_groupped = A.groupby(['random_strats']).yes.agg(["var", weight])
    random_strat_var = (random_groupped['var'] * random_groupped['weight']).dropna().sum()
    A.drop(columns=['random_strats'])
    bad_strats += A.yes.var() - random_strat_var < 0
print('Разбиений на страты, увеличивших дисперсию: {}/{}'.format(bad_strats, c))

Разбиений на страты, увеличивших дисперсию: 12/20


## CUPED — Controlled-experiment Using Pre-Experiment Data

$Y'_i = Y_i - (X_i - \mu_X) \Theta$

$Y' = \overline Y - (\overline X - \mu_X) \Theta$
* $Y$ — метрика во время проведения эксперимента
* $X$ — метрика до проведения эксперимента
* $\Theta = \frac{cov(X, Y)}{var(X)}$

$var(Y_{CUPED}) = var(\overline Y - \Theta \overline X) = \frac{1}{n}var(Y - \Theta X) = \frac{1}{n}[var(Y) + \Theta^2 var(X) - 2 \Theta cov(Y, X)]$

Выбор $\Theta$ таков, что дисперсия минимизируется.

$var(Y_{CUPED}) = \frac{1}{n}(var(Y) + \frac{cov^2(Y, X)}{var(X)} - 2 \frac{cov^2(Y, X)}{var(X)}) = \frac{1}{n}(var(Y) - \frac{cov^2(Y, X)}{var(X)}) =
\frac{1}{n}var(Y)(1 - \frac{cov^2(Y, X)}{var(X)var(Y)}) = var(\overline Y)(1 - cor^2(X, Y))$

$$\frac{var(Y_{CUPED})}{var(\overline Y)} = (1 - cor^2(X, Y))$$

Оценка CUPED **несмещенная** за счёт вычитания $\mu_X$

$ATE$ — Average Treatment Effect<br>
$ATE = Y_{B_{CUPED}} - Y_{A_{CUPED}}$

Можно использовать в качестве X что угодно, сокращающее $1 - cor^2(X, Y)$

CUPAC — Controlled-experiment Using Prediction As Covariate